# Biostat 257 Homework 3

**Due May 15 @ 11:59PM**

We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

In [1]:
versioninfo()

Julia Version 1.4.1
Commit 381693d3df* (2020-04-14 17:20 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)
Environment:
  JULIA = D:\Julia-1.4.0\bin


## Q1. (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.\
**Solution:** The elements of the transition matrix is the probability (s)he goes to the next page j, so we know:
$$
\begin{eqnarray*}
	P_{ij}= \begin{cases}
	\frac{p}{r_i} + \frac{1-p}{n} & \text{if $r_i>0$} \\
	\frac{1}{n} & \text{$r_i=0$}
	\end{cases}.
\end{eqnarray*}
$$
And we can decompose the transition Matrix $\mathbf{P}$ as $\mathbf{P = DA + Z1_n'}$ where D is a $\mathbf{n\times n}$ diagonal matrix with diagonal element $\mathbf{(i,i)}$ being $\mathbf{\frac{p}{r_i}}$ if $\mathbf{r_i>0}$ and $\mathbf{0}$ if $\mathbf{r_i=0}$. $\mathbf{Z}$ is a $\mathbf{n\times 1}$ vector with i-th element $\mathbf{\frac{1-p}{n}}$ if $\mathbf{r_i>0}$ and $\mathbf{\frac{1}{n}}$ if $\mathbf{r_i=0}$.

## Q2. Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.
**Solution:** We can cast the function as a linear system problem by rearrangement:
$$\mathbf{P^TX} = \mathbf{X}$$
$$\mathbf{(I-P^T)X} = 0$$

## Q3. (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [3]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

include group.jl for user defined matrix generators
verify download of index files...
used remote site is https://sparse.tamu.edu/?per_page=All
populating internal database...
downloading: https://sparse.tamu.edu/MM/SNAP/web-Google.tar.gz
web-Google/web-Google.mtx


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [4]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool,Int64} with 5105039 stored entries:
  [11343 ,      1]  =  1
  [11928 ,      1]  =  1
  [15902 ,      1]  =  1
  [29547 ,      1]  =  1
  [30282 ,      1]  =  1
  [31301 ,      1]  =  1
  [38717 ,      1]  =  1
  [43930 ,      1]  =  1
  [46275 ,      1]  =  1
  [48193 ,      1]  =  1
  [50823 ,      1]  =  1
  [56911 ,      1]  =  1
  ⋮
  [608625, 916427]  =  1
  [618730, 916427]  =  1
  [622998, 916427]  =  1
  [673046, 916427]  =  1
  [716616, 916427]  =  1
  [720325, 916427]  =  1
  [772226, 916427]  =  1
  [785097, 916427]  =  1
  [788476, 916427]  =  1
  [822938, 916427]  =  1
  [833616, 916427]  =  1
  [417498, 916428]  =  1
  [843845, 916428]  =  1

Compute summary statistics:
* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  

In [3]:
@show Base.summarysize(A)
@show typeof(A[1,1])

Base.summarysize(A) = 53276943
typeof(A[1, 1]) = Bool


Bool

**Solutions:** Currently it takes 53276943 bytes (around 53MB), and if converted the type from Boolean (1bytes) to Float 64(8bytes), the memory size will be increased by 8 times.

In [5]:
@show sum(sum(A))
@show count(i->(i==0), sum(A, dims=2))

sum(sum(A)) = 5105039
count((i->begin
            #= In[5]:2 =#
            i == 0
        end), sum(A, dims = 2)) = 176974


176974

* number of web pages
* number of edges (web links). 
* number of dangling nodes (pages with no out links)

**Solutions:**  According to the matrix size and stored entries number, we know there are **916428** web pages, and **5105039** edges, and **176974** dangling nodes

* histogram of in-degrees  
* list the top 20 pages with the largest in-degrees?  

In [46]:
using UnicodePlots
inD = vec(sum(A, dims=1))
@show histogram(inD,nbins = 25 ,xscale = log10)
rankInD = sortperm(inD)
@show findall(x->x<=20, rankInD)

histogram(inD, nbins = 25, xscale = log10) =                     ┌                                        ┐ 
   [   0.0,  500.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 916114   
   [ 500.0, 1000.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇ 180                          
   [1000.0, 1500.0) ┤▇▇▇▇▇▇▇▇ 32                               
   [1500.0, 2000.0) ┤▇▇▇▇▇▇▇ 20                                
   [2000.0, 2500.0) ┤▇▇▇▇▇▇ 16                                 
   [2500.0, 3000.0) ┤▇▇▇▇▇▇▇ 20                                
   [3000.0, 3500.0) ┤▇▇▇▇▇▇▇ 18                                
   [3500.0, 4000.0) ┤▇▇▇▇▇ 10                                  
   [4000.0, 4500.0) ┤▇▇▇▇▇ 9                                   
   [4500.0, 5000.0) ┤▇▇▇ 4                                     
   [5000.0, 5500.0) ┤▇▇▇ 4                                     
   [5500.0, 6000.0) ┤ 0                                        
   [6000.0, 6500.0) ┤ 1                                        
                    └                                      

20-element Array{Int64,1}:
      1
      2
      3
 201884
 201885
 201886
 201887
 201888
 201889
 201890
 488779
 603321
 603322
 698817
 726686
 830133
 844453
 844454
 901699
 915320

* histogram of out-degrees
* which the top 20 pages with the largest out-degrees?

In [47]:
outD = vec(sum(A, dims=2))
@show histogram(outD,nbins = 25 ,xscale = log10)
rankOutD = sortperm(outD)
@show findall(x->x<=20, rankOutD)

histogram(outD, nbins = 25, xscale = log10) =                   ┌                                        ┐ 
   [  0.0,  20.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 891798   
   [ 20.0,  40.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 22628             
   [ 40.0,  60.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 1329                    
   [ 60.0,  80.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 371                        
   [ 80.0, 100.0) ┤▇▇▇▇▇▇▇▇▇▇▇ 124                           
   [100.0, 120.0) ┤▇▇▇▇▇▇▇▇▇▇ 86                             
   [120.0, 140.0) ┤▇▇▇▇▇▇▇▇ 29                               
   [140.0, 160.0) ┤▇▇▇▇▇▇ 15                                 
   [160.0, 180.0) ┤▇▇▇▇▇▇ 15                                 
   [180.0, 200.0) ┤▇▇▇▇▇ 8                                   
   [200.0, 220.0) ┤▇▇▇▇▇ 7                                   
   [220.0, 240.0) ┤▇▇▇ 3                                     
   [240.0, 260.0) ┤▇▇▇▇▇ 8                                   
   [260.0, 280.0) ┤▇▇▇ 3                                     
   [280.0, 300.0) ┤ 0   

20-element Array{Int64,1}:
      1
      2
 176975
 176976
 304912
 400165
 400166
 466141
 466142
 607465
 607466
 607467
 715809
 744905
 744906
 744907
 844414
 857484
 869262
 891799

* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

In [60]:
spy(A[1:10000, 1:10000])

                      Sparsity Pattern
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄⠀⠀⡀⠈⠀⠀⠙⠀⠀⠀⢀⢀⠂⠀⠀⡄⢀⠀⠠

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

## Q4. (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer.

**Solutions:** 
1. **For LU approach**, it will need to store n by n matrix for coefficient and n vector for the results, which will need in total of $916428^2*8+916428*8= 6718.73\mathrm{GB}$. Assume we can do $10^{12}$ flops per second and it will take $(2\times(916428)^3/3+ 2\times(916428)^2)/10^{12}= 513103$ second $\approx$ 5.94 days

## Q5. Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide is methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [5]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    oneV      :: Vector{T}
    R         :: Vector{T}
    d         :: Vector{T}
    z         :: Vector{T}
    ztv       :: Vector{T}
    dtv       :: Vector{T}
    Atdtv     :: Vector{T}
    out       :: Vector{T}
    
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    
    oneV = ones(n)
    # TODO: initialize and pre-allocate arrays
    # R = vec(sum(A, dims=2))
    R = A*ones(n)
    
    # Pre-allocate and cal decomposed matrix P
    d = Vector{T}(undef,n)
    z = Vector{T}(undef,n)
    for i in 1:n
        if R[i] == 0
            d[i], z[i] = 0, 1/n
        else
            d[i], z[i] = telep/R[i], (1-telep)/n
        end
    end
    
    # changing Vectors for updating Mv
    dtv = similar(d)
    Atdtv = similar(d)
    ztv = similar(z)
    out = similar(z)
    
    PageRankImPt(A, telep, oneV, R, d, z, ztv, dtv, Atdtv, out)
end

PageRankImPt

In [112]:
?rmul!

search: rmul! baremodule parentmodule promote_rule



```
rmul!(A::AbstractArray, b::Number)
```

Scale an array `A` by a scalar `b` overwriting `A` in-place.  Use [`lmul!`](@ref) to multiply scalar from left.  The scaling operation respects the semantics of the multiplication [`*`](@ref) between an element of `A` and `b`.  In particular, this also applies to multiplication involving non-finite numbers such as `NaN` and `±Inf`.

!!! compat "Julia 1.1"
    Prior to Julia 1.1, `NaN` and `±Inf` entries in `A` were treated inconsistently.


# Examples

```jldoctest
julia> A = [1 2; 3 4]
2×2 Array{Int64,2}:
 1  2
 3  4

julia> rmul!(A, 2)
2×2 Array{Int64,2}:
 2  4
 6  8

julia> rmul!([NaN], 0.0)
1-element Array{Float64,1}:
 NaN
```

---

```
rmul!(A, B)
```

Calculate the matrix-matrix product $AB$, overwriting `A`, and return the result. Here, `B` must be of special matrix type, like, e.g., [`Diagonal`](@ref), [`UpperTriangular`](@ref) or [`LowerTriangular`](@ref), or of some orthogonal type, see [`QR`](@ref).

# Examples

```jldoctest
julia> A = [0 1; 1 0];

julia> B = LinearAlgebra.UpperTriangular([1 2; 0 3]);

julia> LinearAlgebra.rmul!(A, B);

julia> A
2×2 Array{Int64,2}:
 0  3
 1  2

julia> A = [1.0 2.0; 3.0 4.0];

julia> F = qr([0 1; -1 0]);

julia> rmul!(A, F.Q)
2×2 Array{Float64,2}:
 2.0  1.0
 4.0  3.0
```


In [6]:
LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
Base.getindex(M::PageRankImPt, i, j) = M.telep
# implement the basic function for struct PageRankImPt
function Base.getindex(M::PageRankImPt, i, j)
    if i==j
        return (1-M.A[i,j] * M.d[i] - M.z[i])
    else
        return (-M.A[i,j] * M.d[i] - M.z[i])
    end
end

In [13]:
# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}) where T <: AbstractFloat
    # TODO: implement mul!(out, M, v)
    #D^Tv = > A^TD^Tv
    M.dtv .= M.d .* v
    mul!(M.Atdtv, transpose(A), M.dtv)
    
    #1z^Tv
    M.ztv .= dot(M.z, v).*M.oneV
    
    out .= v .- M.ztv .- M.Atdtv
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    # TODO: implement mul!(out, transpose(M), v)
    mul!(M.dtv, M.A, v) # actually calculate AV here
    M.Atdtv .= M.d .* M.dtv  # actually calculate DAV 
    
    M.ztv .= M.z .* (sum(v).*M.oneV) # actually calculate Z1'V
    # out variable will be re-allocated when calling the function
    # and thus no need to pre-allocate in the structure
    out .= v .- M.Atdtv .- M.ztv
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M}^T \mathbf{x} = \mathbf{x}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv`. **Do not put this file in your Git**. You can add a line `pgrksol.csv` to your `.gitignore` file.

In [8]:
using DelimitedFiles

isfile("pgrksol.csv") || download("https://raw.githubusercontent.com/ucla-biostat-257-2020spring/ucla-biostat-257-2020spring.github.io/master/hw/hw3/pgrksol.csv")
xsol = vec(readdlm("pgrksol.csv"));

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [14]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

# @assert transpose(M) * ones(n) ≈ zeros(n)
# Use the relaxed criteria
@assert norm(transpose(M) * ones(n)) < 1e-12

In [15]:
# @assert M * xsol ≈ zeros(n)
# Use the relaxed criteria
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate litte memory.

In [16]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     23.605 ms (0.00% GC)
  median time:      25.278 ms (0.00% GC)
  mean time:        26.377 ms (0.00% GC)
  maximum time:     29.865 ms (0.00% GC)
  --------------
  samples:          184
  evals/sample:     1

In [17]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     24.118 ms (0.00% GC)
  median time:      24.468 ms (0.00% GC)
  mean time:        24.731 ms (0.00% GC)
  maximum time:     29.312 ms (0.00% GC)
  --------------
  samples:          195
  evals/sample:     1

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [18]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

19.84

**Hint**: My median run times are 30-40 ms and memory allocations are 0-16 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [19]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n);

In [20]:
# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 

([3.378375565730854e-5, 2.0710008290004846e-6, 3.663160494081563e-6, 7.527477309596129e-7, 8.63326175148559e-7, 1.7694050004451021e-6, 2.431230085065255e-7, 6.368398928398891e-7, 4.7449719187525534e-7, 2.6895483443481086e-7  …  3.1679174389517645e-6, 9.72731264250764e-7, 6.588685596562533e-7, 7.736991169983615e-7, 2.431230085065255e-7, 1.6219392163585108e-6, 3.912137945773289e-7, 2.431230085065255e-7, 7.296031020719451e-6, 6.330963730931431e-7], ConvergenceInfo: no converged values after 1 iterations and 31 applications of the linear map;
norms of residuals are given by (2.563520017593024e-8,).
)

In [21]:
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(([3.378342822183504e-5, 2.071015539245162e-6, 3.663065985231259e-6, 7.527510785590286e-7, 8.63328599714724e-7, 1.7694182527349206e-6, 2.4312303829080455e-7, 6.36841718073685e-7, 4.7449737037420244e-7, 2.689548611092025e-7  …  3.1679469739726655e-6, 9.726880410215637e-7, 6.588614478531613e-7, 7.737011774702039e-7, 2.4312303829080455e-7, 1.6219204214220026e-6, 3.912130060624972e-7, 2.4312303829080455e-7, 7.29603383131291e-6, 6.330939996667897e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821171057225167e-13,).
), 4.893589299, 1010851097, 0.158265197, Base.GC_Diff(1010851097, 137, 0, 123784, 6, 0, 158265197, 4, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [22]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [23]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 8 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [24]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

((Complex{Float64}[-4.51831843083042e-15 + 0.0im], Array{Complex{Float64},1}[[0.0056358269538073135 + 0.0im, 0.00034549143807828867 + 0.0im, 0.000611080849412517 + 0.0im, 0.00012557561626035086 + 0.0im, 0.00014402240532803155 + 0.0im, 0.00029517830503984565 + 0.0im, 4.0558328286778296e-5 + 0.0im, 0.00010623935784851998 + 0.0im, 7.915671116195223e-5 + 0.0im, 4.486765066724855e-5 + 0.0im  …  0.0005284839899797004 + 0.0im, 0.00016226599147988716 + 0.0im, 0.00010991273837609647 + 0.0im, 0.00012907055855751584 + 0.0im, 4.0558328286778296e-5 + 0.0im, 0.00027057238743203937 + 0.0im, 6.526302748333306e-5 + 0.0im, 4.0558328286778296e-5 + 0.0im, 0.0012171406601327998 + 0.0im, 0.0001056141551069903 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.575962515146957e-14,).
), 9.648659201, 1584017541, 0.1799063, Base.GC_Diff(1584017541, 213, 0, 139424, 51, 0, 179906300, 4, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [25]:
@assert abs(Real(vals[1])) < 1e-8

In [32]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [27]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 11 seconds.

## Q6. (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees?

**Solution:** 
The rank number is extracted by using`sortperm` and find the corresponding row number for rank smaller than 20 (top20), which is shown below and different from both in-degrees and out-degrees. Only 2~3 website index occur in all these list.

In [51]:
orderI = sortperm(x_arnoldi)
@show findall(x->x<=20, orderI) 
@show findall(x->x<=20, rankInD)
@show findall(x->x<=20, rankOutD)

findall((x->begin
            #= In[51]:2 =#
            x <= 20
        end), orderI) = [1, 2, 3, 249292, 295651, 311132, 313180, 413532, 527221, 530130, 618065, 656417, 661692, 692881, 768969, 816429, 836458, 871154, 878833, 914688]
findall((x->begin
            #= In[51]:3 =#
            x <= 20
        end), rankInD) = [1, 2, 3, 201884, 201885, 201886, 201887, 201888, 201889, 201890, 488779, 603321, 603322, 698817, 726686, 830133, 844453, 844454, 901699, 915320]
findall((x->begin
            #= In[51]:4 =#
            x <= 20
        end), rankOutD) = [1, 2, 176975, 176976, 304912, 400165, 400166, 466141, 466142, 607465, 607466, 607467, 715809, 744905, 744906, 744907, 844414, 857484, 869262, 891799]


20-element Array{Int64,1}:
      1
      2
 176975
 176976
 304912
 400165
 400166
 466141
 466142
 607465
 607466
 607467
 715809
 744905
 744906
 744907
 844414
 857484
 869262
 891799

## Q7. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.